<h1><span style="color:blue">Toward a warranty of micro-simulators for mixed traffic assessement: <br> 
    can we pass the prerequirement of test-cases definition? </span></h1> 

CLUMITISI: Clustering Mixed Traffic Situations <br> 
@author: Thibault Charlottin <br>
@contributor: Christine Buisson <br>
Licit-Eco7, 2022

This script is devoted to realize simulations corresponding to a wide variety of freeway mixed (Automated Cruise Control and Human Driven Vehicles) traffic situations and to cluster them with three classical clustering methods. The script is working on top of the Sumo code that you must install (in the Windows version) before processing the current script.

The first part allows you to create simulations on SUMO with two car-following models and to compute them. In the second and third parts you can analyse and cluster obtained data.

It was developped by Thibault Charlottin during his master internship April-August 2022, under the supervision of Christine Buisson in the LICIT-ECO7 Lab, in Lyon, France. https://licit-lyon.eu/

It is divided into parts:<br>
- 1st part: file simulation using Traci module 
- 2nd part: indicators definition and computation
- 3rd part: indicator clustering and plot editing<br>

SUMO can be downloaded for windows version (the only one working with this code) at
    https://sumo.dlr.de/docs/Downloads.php

This source is provided upon the Eclipse Public License 2.0 (see: https://www.eclipse.org/legal/epl-2.0/)

In the following, the comments are in English but some of the variables names are in French.


**Warning #1 the Traci librairies are Windows based, the simulation section won't work on Mac**<br>
**Warning #2 this code not tested on Linux distributions**


## <span style="color:blue"> Setup : libraires and paths</span>


In [1]:
#imports

#SUMO libraries ! NOTE THAT YOU CANNOT USE THOSE LIBRARIES ON MAC !
import traci
import traci.constants as tc
import pysumo as pysumo

#handle data libraries
import pandas as pd
import numpy as np

#plots libraries
import seaborn as sns
import itertools
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.colors import LogNorm
import plotly.express as px

#machine learning libraries
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import spectral_clustering
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

#misc libraries
import random as rd
import os
from os import listdir
from os.path import isfile, join
import glob
import sys
import time
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from time import time
from multiprocessing import Process, Pool
from alive_progress import alive_bar
from numba import jit, cuda
import warnings

In [2]:
#set plot style
sns.set_theme()
%matplotlib inline
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 
warnings.filterwarnings('ignore') #to disable warnings in the notebook

In [3]:
#global path TO BE CHANGE ACCORDINGLY TO YOUR WORKPATH
#we advise you to use this path to store all the files you will use and create during this script
global_path = "C:/Users/tchar/Downloads/simulation files and code/"

In [4]:
#SUMO path
#mandatory for using the Traci module
sumoBinary = "C:/Program Files (x86)/Eclipse/Sumo/bin/sumo.exe"

### <span style="color:green"> Import the input files</span>

In [5]:
#opening csv files that contains the names of all files
df_section_Krauss = pd.read_csv(global_path+"inputs/inputs Krauss.csv", delimiter=';')
df_section_EIDM = pd.read_csv(global_path+"inputs/inputs EIDM.csv", delimiter=';')


In [6]:
#deleting all simulation files that might be in the folder so that we get an updated version
models_list = ['krauss','EIDM']
for k in models_list :
    dir = global_path+'simulations/'+ k +'/'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

In the following cell, please note that the lines within the 'lines to change' are to be modified accordingly to the local path you used for the downloaded input files

In [15]:
#function that creates the sumocfg files
#please note that the lines within the 'lines to change' are to be modified accordingly to your local path
def export_sumogfg(df,geometry, replication, model):
    for k in range(len(df['combis géométrie'])):
        if df['combis géométrie'][k]!='NaN':
            for l in range(len(df['combis demande'])):
                #changing filenames to the imposed SUMO format
                geom = str(df['combis géométrie'][k])  #network files             
                traf = str(df['combis demande'][l])    #route files
                geom = geom.replace(".net.xml", "")   #cleaning the names
                traf = traf.replace(".rou.xml", "")   #cleaning the names
                doc_out=global_path+'/simulations/'+ model +'/' +geom+'_'+traf+''+model+'.sumocfg' #export path
                #writting the XML file 
                xml=['<configuration xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://sumo.dlr.de/xsd/sumoConfiguration.xsd\">']
                xml.append('\n  <input>')
                #defining the path for input files
                #******LINES TO CHANGE ACCORDINGLY TO YOUR LOCAL PATH******
                xml.append('\n    <net-file value=\"' + "C:"+"\\"+"Users"+"\\"+"tchar"+"\\"+"Downloads"+"\\"+"simulation files and code"+"\\"+"inputs"+"\\" +model+'\\'  + 'network' + '\\' + str(df['combis géométrie'][k]) + '\"/>')
                xml.append('\n    <route-files value=\"' + "C:"+"\\"+"Users"+"\\"+"tchar"+"\\"+"Downloads"+"\\"+"simulation files and code"+'\\'+"inputs"+"\\"+model+'\\'  +  'route' + '\\' + str(df['combis demande'][l]) + '\"/>')
                #****** END OF LINES TO CHANGE ACCORDINGLY TO YOUR LOCAL PATH******
                xml.append('\n  </input>')
                xml.append('\n  ')
                #defining the simulation lenght and the timestep
                xml.append('\n  <time>')
                xml.append('\n    <begin value=\"0\"/>')
                xml.append('\n    <end value="3600"/>')
                xml.append('\n    <step-length value="1"/>')
                xml.append('\n  </time>')             
                xml.append('\n\t</configuration>')
                with open(doc_out, 'w') as f:  # Writing the .sumocgf file
                    for line in xml:
                        f.write(line)
    return

In [16]:
export_sumogfg(df_section_EIDM,'section', 10,'EIDM')
export_sumogfg(df_section_Krauss,'section', 10,'krauss')

In [17]:
#some files does not correspond to anything due to the shape of the csv file let's delete them
models = ['EIDM','krauss']
for k in models :
    files = os.listdir(global_path+'/simulations/' + k + '/')
    for file_name in files:
    # construct full file path
        filename = os.path.join(global_path+'/simulations/' + k + '/', file_name)
        if 'nan' in filename :
            os.remove(os.path.join(dir, filename))

## <span style="color:blue"> First part: simulating</span>


This part use the .sumocfg files we just created as inputs <br>
output is a dataframe for each simulation containing car ID, car lenght lattitude, logitude , speed, acceleration for each timestep <br>

In [18]:
simulation_path_section_krauss = global_path + "simulations/krauss/"

In [19]:
simulation_path_section_EIDM = global_path + "simulations/EIDM/"

In [20]:
#function to run simulation
def running_simulation(simulation_file, simulation_lenght,path_out, sim_name, replication_number):
    sumoCmd = [sumoBinary, "-c", simulation_file,"--start","--quit-on-end", '--random', '--no-step-log']
    step = 0
    #defining variables to be save
    timestep = []
    IDs = []
    speed = []
    acceleration = []
    x_position = []
    y_position = []
    lane = []
    lenght = []
    time_start = time()
    traci.start(sumoCmd)
    for j in range(simulation_lenght):
        #executting SUMO
        traci.simulationStep()
        at_t_IDs = traci.vehicle.getIDList()
        at_t_IDs= [*at_t_IDs]
        at_t_positions = [traci.vehicle.getPosition(id) for id in traci.vehicle.getIDList()]
        at_t_acceleration = [traci.vehicle.getAcceleration(id) for id in traci.vehicle.getIDList()]      
        at_t_lane = [traci.vehicle.getLaneID(id) for id in traci.vehicle.getIDList()]
        at_t_speed = [traci.vehicle.getSpeed(id) for id in traci.vehicle.getIDList()]
        at_t_lenght = [traci.vehicle.getLength(id) for id in traci.vehicle.getIDList()]
        #traci doesn't save the variables we have do it ourselve
        for k in range(len(at_t_speed)) : 
        #saving data for each timestep
            timestep.append(j)
            IDs.append(at_t_IDs[k])
            x_position.append(at_t_positions[k][0])
            y_position.append(at_t_positions[k][1])
            acceleration.append(at_t_acceleration[k])
            speed.append(at_t_speed[k])
            lenght.append(at_t_lenght[k])
            lane.append(at_t_lane[k])
    traci.close()
    #saving simualtion result in a dataframe
    df = pd.DataFrame({'timestep' : timestep, 'ID' : IDs,'lenght' : lenght,'lane': lane,'x' : x_position,'y' : y_position, 'acceleration' : acceleration, 'speed' : speed})
    time_end = time()
    df.to_csv(path_out+' '+ str(sim_name) + 'EIDM model replication num '+str(replication_number)+'.csv')
    return

In [21]:
#to run the simulation at the scale of a whole folder
def simulating_all_the_files (path,path_out,simulation_lenght) : 
    fichiers = [f for f in listdir(path) if isfile(join(path, f))]
    counter = 0
    #on simule tous les .sumocfg du dossier
    for filename in glob.glob(os.path.join(path, '*.sumocfg')):
            for k in tqdm_notebook(range(10), desc = 'simulating' + str(fichiers[counter])):
                running_simulation (filename, 3600, path_out, fichiers[counter], k)
            print('scenario ' + str(fichiers[counter]) + ' has been fully simulated 10 times')
            counter += 1
    return 

In [22]:
#simulating for the sections with EIDM model
#we run 10 replication with a 1h lenght 
path_out = global_path + "output/Krauss/"
simulating_all_the_files(simulation_path_section_krauss,path_out,3600)

simulatingReseau section 3 voies brouillard_2200 0 ACCkrauss.sumocfg:   0%|          | 0/10 [00:00<?, ?it/s]

TraCIException: Connection 'default' is already active.

In [ ]:
#simulating for the merging with Krauss Model
#we run 10 replication with a 1h lenght 
path_out = global_path + "output/EIDM/"
simulating_all_the_files(simulation_path_section_EIDM,path_out,3600)

## <span style="color:blue"> Second part: computing indicators</span>


### <span style="color:green"> Import results</span>

In [ ]:
#defining results path
results_path_section_Krauss = global_path + "output/Krauss/"
results_path_section_EIDM = global_path + "output/EIDM/"


In [ ]:
def plot_car_trajectories(dataframe,variables, car_number_list, color_list,namelist):
    #to visualise one's car trajectory from his network injection to his network departure
    palette = itertools.cycle(sns.color_palette()) 
    for k in range(len(car_number_list)) :
        fig, axs = plt.subplots(len(car_number_list[k]), len(variables),figsize=(60,60)) #creating a subplot
        for i in range(len(car_number_list[k])):
            filtered_df = dataframe[dataframe.ID == car_number_list[k][i]]
            c = next(palette) 
            for j in range(len(variables)) :
                legend = "veh name = " + str(car_number_list[k][i])
                
                axs[i,j].plot(filtered_df['timestep'],filtered_df[variables[j]], color = c, linewidth=10)
                axs[i,j].set_xlabel('time (s)', fontsize = 50.0)
                axs[i,j].set_ylabel(namelist[j], fontsize = 50)
                axs[i,j].set_title( "car: " + str(car_number_list[k][i]) + " " + namelist[j] + ' over time', fontsize = 60)
                axs[i,j].tick_params(axis='x',labelsize=40)
                axs[i,j].tick_params(axis='y',labelsize=40)
                
    plt.show()
    return

In [ ]:
#cell to plot some trajectories
#car-number_list = [f_0.0]
#color_list = ['blue']
#variables = ['speed']
#namelist = = ['speed']

### <span style="color:green"> Creating new indicators</span>

We create 3 indicators: <br>
- spacing
- intervehicular time
- time to Collision <br>
please refer to the article for more details

In [ ]:
def compute_traffic_indicators(DataFrame) : 
    DataFrame = DataFrame.sort_values(['timestep','x'],ascending=True) #sorting dataframe by position for each car for each timestep
    #importing results that we need to compute them into indicators
    speed = DataFrame['speed']
    ID = DataFrame['ID']
    lane = DataFrame['lane'] 
    timestep = DataFrame['timestep']
    lenght = DataFrame['lenght']
    acceleration = DataFrame['acceleration']
    x = DataFrame['x']
    y = DataFrame['y']
    #creating lists to save computed indicators
    spacing_list = [np.nan]
    IVT_list =[np.nan]
    for k in tqdm_notebook(range(1,len(ID))):
        #compute Traffic indicators (based on n-1th car)
        if lane[k] == lane[k-1] and timestep[k] == timestep[k-1] :
            space = np.sqrt((abs(x[k-1]-x[k]))**2 + ((abs(y[k-1]-y[k]))**2))
            spacing_list.append(space)
            if  speed[k-1]>speed[k]:
                TTC_list.append(space/(speed[k-1]-speed[k]))
            else : 
                TTC_list.append(np.nan)
        else : 
            spacing_list.append(np.nan)
            TTC_list.append(np.nan)
    DataFrame['spacing'] = spacing_list
    DataFrame['TTC'] = TTC_list
    return DataFrame

In [ ]:
def compute_20ile(DataFrame, out_DataFrame_spacing,out_DataFrame_IVT,out_DataFrame_TTC, out_DataFrame_power , out_name) :
    #transforming indicators distributions into twentiles
    #by default the twentile will be the only indicators results saved for RAM efficiency
    DataFrame = compute_traffic_indicators(DataFrame) #create the twentiles
    spacing_list = DataFrame['spacing']
    TTC_list = DataFrame['TTC']
    spacing_20ile = np.nanpercentile(spacing_list, np.arange(0, 100, 5))
    TTC_20ile = np.nanpercentile(TTC_list, np.arange(0, 100, 5))
    out_DataFrame_spacing[out_name] = spacing_20ile
    out_DataFrame_IVT[out_name] = IVT_20ile
    out_DataFrame_TTC[out_name] = TTC_20ile
    out_DataFrame_power[out_name] = power_20ile
    return out_DataFrame_spacing, out_DataFrame_IVT, out_DataFrame_TTC, out_DataFrame_power

In [ ]:
out_DataFrame_spacing= pd.DataFrame()
out_DataFrame_IVT = pd.DataFrame()
out_DataFrame_TTC = pd.DataFrame()
out_DataFrame_power = pd.DataFrame()
paths = [results_path_section_Krauss]
#computing Krauss model section twentiles indicators 
for k in paths : 
    files = os.listdir(k)
    lenght = len(files)
    for file_name in tqdm_notebook(range(lenght)):
        df = pd.read_csv(k+files[file_name])
        print(files[file_name])
        out_DataFrame_spacing, out_DataFrame_IVT, out_DataFrame_TTC, out_DataFrame_power  =compute_20ile(df, out_DataFrame_spacing,out_DataFrame_IVT,out_DataFrame_TTC ,out_DataFrame_power, files[file_name] )
DataFrame_spacing_Krauss = out_DataFrame_spacing.copy()
DataFrame_IVT_Krauss = out_DataFrame_IVT.copy()
DataFrame_TTC_Krauss = out_DataFrame_TTC.copy()
DataFrame_power_Krauss = out_DataFrame_power.copy()

In [ ]:
out_DataFrame_spacing= pd.DataFrame()
out_DataFrame_IVT = pd.DataFrame()
out_DataFrame_TTC = pd.DataFrame()
out_DataFrame_power = pd.DataFrame()
paths = [results_path_section_EIDM]
#computing EIDM model section twentiles indicators 
for k in paths : 
    files = os.listdir(k)
    lenght = len(files)
    for file_name in tqdm_notebook(range(lenght)):
        df = pd.read_csv(k+files[file_name])
        print(files[file_name])
        out_DataFrame_spacing, out_DataFrame_IVT, out_DataFrame_TTC, out_DataFrame_power  =compute_20ile(df, out_DataFrame_spacing,out_DataFrame_IVT,out_DataFrame_TTC ,out_DataFrame_power, files[file_name] )
DataFrame_spacing_EIDM = out_DataFrame_spacing.copy()
DataFrame_IVT_EIDM = out_DataFrame_IVT.copy()
DataFrame_TTC_EIDM = out_DataFrame_TTC.copy()
DataFrame_power_EIDM = out_DataFrame_power.copy()

## <span style="color:blue"> Third part: clustering </span>


### <span style="color:green"> DBscan clustering </span>

First we compute a DBscan clustering

In [ ]:
def DBscan_clustering(DataFrame, minsample, eps) :
    DataFrame = DataFrame.transpose()
    X = DataFrame.values
    dbscan_cluster = DBSCAN(eps=eps, min_samples=minsample)
    dbscan_cluster.fit(X)
    labels = dbscan_cluster.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    results = dbscan_cluster.labels_
    return results, n_clusters_, n_noise_

In [ ]:
def plot_based_on_epsilon(eps_list, df_list, name_columns, indicator_name, tick_list):
    names = list(df_list[0].T.index)
    DB_cluster_DataFrame= pd.DataFrame(names)
    DB_cluster_DataFrame.columns = ['scenario']
    nb_minsample_list = [100,100]
    cluster_nb_list_EIDM = []
    noise_list_EIDM = []
    cluster_nb_list_Krauss = []
    noise_list_Krauss = []
    for l in range (len(eps_list)):
        for k in range (len(df_list)) : 
            results, n_clusters_, n_noise_ = DBscan_clustering(df_list[k], nb_minsample_list[k], eps_list[l])
            if k == 0:
                cluster_nb_list_EIDM.append(n_clusters_)
                noise_list_EIDM.append(n_noise_/len(df_list[k].columns)*100)
            else : 
                cluster_nb_list_Krauss.append(n_clusters_)
                noise_list_Krauss.append(n_noise_/len(df_list[k].columns)*100)
    fig, ax = plt.subplots(2, 1,sharex=True,figsize=(30,30))
    ax[0].plot(eps_list,cluster_nb_list_EIDM,linewidth = 5, label = 'EIDM model')
    ax[0].plot(eps_list,cluster_nb_list_Krauss,':',linewidth = 5, label = 'Krauss model')
    ax[0].legend( prop={'size': 40})
    ax[0].set_ylabel('number of clusters', size = 35)    
    ax[0].set_title(' number of '+ indicator_name + ' clusters using DBscan depending on the epsilon value', size = 45)
    ax[0].set_yticklabels(tick_list, fontsize = 35)
    ax[1].bar(eps_list,noise_list_EIDM, label = 'EIDM model',width  = 0.5)
    Kraussbars = [x + 0.5 for x in eps_list]
    ax[1].bar(Kraussbars,noise_list_Krauss, label = 'Krauss model',width = 0.5)
    ax[1].set_xlabel('epsilon', size = 35)       
    ax[1].set_ylabel('percentage of outliers', size = 35)    
    ax[1].set_title('number of '+ indicator_name + '  outliers using DBscan depending on the epsilon value', size = 45)
    ax[1].legend( prop={'size': 40})
    plt.xticks(fontsize=35)
    plt.yticks(fontsize=35)
    plt.show()
    return

In [ ]:
df_list = [df_EIDM_spacing, df_Krauss_spacing]
name_columns = ['EIDM spacing','Krauss spacing']
eps_list = [k for k in range(1,45)]
indicator_name = 'spacing'
tick_list = [k for k in range (0,10)]
plot_based_on_epsilon(eps_list, df_list, name_columns, indicator_name, tick_list)

In [ ]:
df_list = [df_EIDM_TTC, df_Krauss_TTC]
name_columns = ['EIDM TTC','Krauss TTC']
eps_list = [k for k in range(1,45)]
indicator_name = 'TTC'
tick_list = [k for k in range (0,10)]
plot_based_on_epsilon(eps_list, df_list, name_columns, indicator_name, tick_list)

### <span style="color:green"> Kmeans clustering </span>

Then we compute Kmeans clustering, to determine the ideal cluster number we use the WCSS elbow method

In [ ]:
def choosing_cluster_number(DataFrame):
    wcss = []
    for i in range(1,11):
        model = KMeans(n_clusters = i, init = 'k-means++')
        model.fit(DataFrame)
        wcss.append(model.inertia_)
    return wcss


In [ ]:
df_list = [DataFrame_EIDM_spacing, DataFrame_EIDM_TTC, DataFrame_Krauss_spacing, DataFrame_Krauss_TTC]
name_columns = ['EIDM spacing', 'EIDM TTC','EIDM spacing', 'EIDM TTC']
names = list(DataFrame_spacing.T.index)
cluster_DataFrame= pd.DataFrame(names)
cluster_DataFrame.columns = ['scenario']
fig, axs = plt.subplots(2, 2,figsize=(30,30)) #creating a subplot 
wcss = choosing_cluster_number(DataFrame_EIDM_spacing)
axs[0,0].plot(list(k for k in range(1,11)), wcss)
axs[0,0].set_xlabel('number of clusters', size = 20)
axs[0,0].set_ylabel('WCSS', size = 20)
axs[0,0].set_title('EIDM spacing', size = 20)
wcss = choosing_cluster_number(DataFrame_EIDM_TTC)
axs[0,1].plot(list(k for k in range(1,11)), wcss)
axs[0,1].set_xlabel('number of clusters', size = 20)
axs[0,1].set_ylabel('WCSS', size = 20)
axs[0,1].set_title('EIDM TTC', size = 20)
wcss = choosing_cluster_number(DataFrame_Krauss_spacing)
axs[1,0].plot(list(k for k in range(1,11)), wcss)
axs[1,0].set_xlabel('number of clusters', size = 20)
axs[1,0].set_ylabel('WCSS', size = 20)
axs[1,0].set_title('Krauss spacing', size = 20)
wcss = choosing_cluster_number(DataFrame_Krauss_TTC)
axs[1,1].plot(list(k for k in range(1,11)), wcss)
axs[1,1].set_xlabel('number of clusters', size = 20)
axs[1,1].set_ylabel('WCSS', size = 20)
axs[1,1].set_title('Krauss TTC', size = 20)

The elbow method gives us 5 clusters to be created for the spacing indicator and 3 for the TTC indicator

In [ ]:
def clustering_data(DataFrame, clusters) : 
    #function to cluster using kmeans algorithm
    model = MiniBatchKMeans(n_clusters= clusters)
    labels = model.fit_predict(DataFrame) #apply kmeans to the twentile dataframe
    results = pd.DataFrame([DataFrame.index,labels]).T
    results.columns = ['scenario', 'label'] #creating a dataframe that computes the cluster assigned to a sample
    centers = np.array(model.cluster_centers_) #exporting kmeans centroids
    closest, _ = pairwise_distances_argmin_min(model.cluster_centers_, DataFrame)
    return results, centers, closest

In [ ]:
df_list = [DataFrame_EIDM_spacing, DataFrame_EIDM_TTC, DataFrame_Krauss_spacing, DataFrame_Krauss_TTC]
name_columns = ['EIDM spacing', 'EIDM TTC','Krauss spacing', 'Krauss TTC']
names = list(DataFrame_EIDM_spacing.T.index)
cluster_DataFrame_kmeans= pd.DataFrame(names)
cluster_DataFrame_kmeans.columns = ['scenario']
nb_cluster_list = [5,3,5,3]#number of clusters to be created
dico_closest = {}
for k in range (len(df_list)) : #clustering the entire set of twentiles for both TTC and spacing
    results, centers, closest = clustering_data(df_list[k],nb_cluster_list[k])
    cluster_DataFrame_kmeans[name_columns[k] + '_cluster']= results['label']
    dico_closest[name_columns[k]] = closest
cluster_DataFrame_kmeans.set_index('scenario',inplace=True)
for k in cluster_DataFrame.columns : 
    cluster_DataFrame_kmeans[k] = cluster_DataFrame_kmeans[k].astype(float)
cluster_DataFrame_kmeans

In [ ]:
def plot_cluster_distribution(indicator, cluster_df, cluster_number, indicator_df) : 
    """"input indicator to be plotted, dataframe that describes the cluster number, list of cluster names
        for this indicator, dataframe that contains the twentiles for this indicator"""
    sample_number = len(cluster_df[indicator+'_cluster']) #computing the importion in percentage of each cluster
    count_df = pd.DataFrame(cluster_df[indicator+'_cluster'].value_counts())
    count_df.sort_index(axis=0, inplace = True)
    count_df['percentage'] = count_df[indicator+'_cluster']/sample_number*100
    y_list = [k for k in range(1,21)]
    plt.rcParams['figure.figsize'] = (20, 15)
    custom_lines = []
    palette = itertools.cycle(sns.color_palette())
    legend_elements = []
    for cluster in cluster_number : 
        color=next(palette)
        df_out = pd.DataFrame()
        for k in tqdm_notebook ( range(len(cluster_df[indicator+'_cluster']))):
            if cluster_df[indicator+'_cluster'][k]==cluster :
                try :
                    liste = list(indicator_df[cluster_df.index[k]])
                    plt.plot(liste, y_list, color = color, alpha=0.3) #plotting the twentiles belonging to the cluster
                except KeyError :
                    continue
        legend_elements.append(Line2D([0], [0], color=color, lw=4, label='cluster ' + str (cluster)+
                                     ' (' + str(round(count_df['percentage'][cluster],2)) + '% of samples)'))
        #creating a legend with the cluster number and its sample share
    plt.ylabel('twentile', size = 25)       
    plt.xlabel('TTC (s)', size = 25)    
    plt.legend(handles=legend_elements, loc='best', prop={'size': 25})
    plt.title(indicator + ' twentile distribution and clusters', size = 45)
    plt.show()
    return 

In [ ]:
plot_cluster_distribution('EIDM spacing',cluster_DataFrame_kmeans , [0, 1, 2, 3 ,4],DataFrame_EIDM_spacing )
plot_cluster_distribution('EIDM TTC',cluster_DataFrame_kmeans , [0, 1, 2],DataFrame_EIDM_TTC)
plot_cluster_distribution('Krauss spacing',cluster_DataFrame_kmeans , [0, 1, 2, 3 ,4],DataFrame_Krauss_spacing)
plot_cluster_distribution('Krauss TTC',cluster_DataFrame_kmeans , [0, 1, 2],DataFrame_Krauss_TTC)


In [1]:
#plot heatmaps
figA = px.density_heatmap(cluster_DataFrame_kmeans, x="EIDM spacing_cluster", y="Krauss spacing_cluster",
                         marginal_x="histogram", marginal_y="histogram",
                          text_auto=True,
                          labels={
                     "EIDM spacing_cluster": "EIDM spacing cluster",
                     "Krauss spacing_cluster": "Krauss spacing cluster"
                 },
                title="heatmap of the spacing clusters"
                        )
figB = px.density_heatmap(cluster_DataFrame_kmeans, x="EIDM TTC_cluster", y="Krauss TTC_cluster",
                         marginal_x="histogram", marginal_y="histogram",
                          text_auto =True,
                          labels={
                     "EIDM TTC_cluster": "EIDM TTC cluster",
                     "Krauss TTC_cluster": "Krauss TTC cluster"
                 },
                title="heatmap of the TTC clusters"
                         )
figA.show()
figB.show()

NameError: name 'px' is not defined

### <span style="color:green"> HCA clustering </span>


We compute a HCA clustering

In [ ]:
#creation of dendrograms to determine the number of clusters we have to compute
# Plot title
plt.title('Hierarchical Clustering Dendrogram EIDM spacing')

# Plot axis labels
plt.xlabel('sample index')
plt.ylabel('distance (Ward)')

# Make the dendrogram
dendrogram(ZsEIDM, labels=df_EIDM_spacing.T.index, leaf_rotation=90)

# Show the graph
plt.show()

In [ ]:
# Plot title
plt.title('Hierarchical Clustering Dendrogram EIDM TTC')

# Plot axis labels
plt.xlabel('sample index')
plt.ylabel('distance (Ward)')

# Make the dendrogram
dendrogram(ZTTCEIDM, labels=df_EIDM_TTC.T.index, leaf_rotation=90)

# Show the graph
plt.show()

In [ ]:
# Plot title
plt.title('Hierarchical Clustering Dendrogram Krauss spacing')

# Plot axis labels
plt.xlabel('sample index')
plt.ylabel('distance (Ward)')

# Make the dendrogram
dendrogram(Zskrauss, labels=df_Krauss_spacing.T.index, leaf_rotation=90)

# Show the graph
plt.show()

In [ ]:
# Plot title
plt.title('Hierarchical Clustering Dendrogram Krauss TTC')

# Plot axis labels
plt.xlabel('sample index')
plt.ylabel('distance (Ward)')

# Make the dendrogram
dendrogram(ZTTCkrauss, labels=df_Krauss_TTC.T.index, leaf_rotation=90)

# Show the graph
plt.show()

In [ ]:
#now we wan cluster the different clusters using the cluster numbers that our dendrograms gave us
df_list = [df_EIDM_spacing, df_EIDM_TTC, df_Krauss_spacing, df_Krauss_TTC]
name_columns = ['EIDM spacing', 'EIDM TTC','Krauss spacing', 'Krauss TTC']
names = list(DataFrame_EIDM_spacing.T.index)
dendrogram_cluster_DataFrame= pd.DataFrame(names)
dendrogram_cluster_DataFrame.columns = ['scenario']
dendrogram_list = [5,7,5,7] #number of clusters to be created based on the dendrograms
dico_closest = {}
for k in range (len(df_list)) : 
    X = df_list[k].T
    cluster = AgglomerativeClustering(n_clusters=dendrogram_list[k], affinity='euclidean', linkage='ward')
    cluster.fit_predict(X)
    dendrogram_cluster_DataFrame[name_columns[k] + '_cluster']= cluster.labels_
dendrogram_cluster_DataFrame.set_index('scenario',inplace=True)
for k in dendrogram_cluster_DataFrame.columns : 
    dendrogram_cluster_DataFrame[k] = dendrogram_cluster_DataFrame[k].astype(float)
dendrogram_cluster_DataFrame

In [ ]:
#plotting heatmaps 
figA = px.density_heatmap(dendrogram, x="EIDM spacing_cluster", y="Krauss spacing_cluster",
                         marginal_x="histogram", marginal_y="histogram",
                          text_auto=True,
                          labels={
                     "EIDM spacing_cluster": "EIDM spacing cluster",
                     "Krauss spacing_cluster": "Krauss spacing cluster"
                 },
                title="heatmap of the spacing clusters for the HCA"
                        )
figB = px.density_heatmap(dendrogram, x="EIDM TTC_cluster", y="Krauss TTC_cluster",
                         marginal_x="histogram", marginal_y="histogram",
                          text_auto =True,
                          labels={
                     "EIDM TTC_cluster": "EIDM TTC cluster",
                     "Krauss TTC_cluster": "Krauss TTC cluster"
                 },
                title="heatmap of the TTC clusters for the HCA"
                         )
figA.show()
figB.show()